In [10]:
pip install pyreadstat

##1.Exploren el diseño de registro de la base de hogar: a priori, ¿qué variables creen pueden ser predictivas de la desocupación y seria útil incluir para perfeccionar el ejercicio del TP3? Mencionen estas variables y justifiquen su elección.



REGION (Región geográfica del hogar): Captura diferencias geográficas significativas en tasas de desempleo, relacionadas con el desarrollo económico local y acceso a oportunidades laborales.

II7 (Tenencia de la vivienda): Refleja estabilidad económica del hogar. Hogares que alquilan pueden estar más presionados económicamente, mientras que aquellos que son propietarios pueden tener más estabilidad.

CH03 (Relación de parentesco con el jefe de hogar): La relación con el jefe de hogar puede influir en la probabilidad de empleo o desempleo, ya que ciertos roles en la familia (como jefe o cónyuge) podrían tener mayores prioridades o responsabilidades económicas.

IV12_3 (Ubicación en villa de emergencia): Refleja una condición de vulnerabilidad socioeconómica que puede limitar el acceso a oportunidades laborales y recursos.

IV6 (Acceso al agua): El acceso al agua (por cañería, fuera del terreno, etc.) indica diferencias en las condiciones materiales del hogar, que podrían correlacionarse con el nivel de ingresos y empleo.

IV11 (Desagüe del baño): El tipo de desagüe refleja el nivel de infraestructura y calidad de vida del hogar, indicadores que pueden influir en la estabilidad laboral.

II8 (Tipo de combustible utilizado para cocinar): Hogares que utilizan gas de red tienen mejor infraestructura en comparación con aquellos que dependen de kerosene, leña o carbón, lo que puede relacionarse con el ingreso y empleo.

CH12 (Nivel educativo más alto alcanzado): La educación es un factor determinante en el acceso al empleo. Los niveles educativos más altos (universitario o posgrado) están directamente relacionados con mejores oportunidades laborales y menores tasas de desempleo.

ITF (Ingreso total familiar): Proporciona una medida global del ingreso familiar, relevante para evaluar la situación económica general del hogar y su relación con el empleo.

IPCF (Ingreso per cápita familiar): Permite evaluar la disponibilidad de recursos económicos por persona en el hogar, siendo un indicador clave de bienestar económico y estabilidad.

##2.Descarguen la base de microdatos de la EPH correspondiente al primer trimestre de 2004 y 2024 en formato .dta y .xls, respectivamente. La base de hogares se llama Hogar_t104.dta y usu_hogar_T124.xls, respectivamente. Eliminen todas las observaciones que no corresponden a los aglomerados de Ciuidad Autónoma de Buenos Aires o Gran Buenos Aires y unan ambos trimestres en una sola base. Esto es, a la base de la encuesta individual de cada año (que usaron en el TP3) unan la base de la encuesta de hogar. Asegúrese de estar usando las variables CODUSU y NRO_Hogar para el merge.


In [89]:
import pyreadstat
import pandas as pd
import requests

# URLs de los archivos
url_2004 = "https://github.com/JorgeJimenezT/Big-Data-UBA---Jimenez-Menendez-Nunez/raw/main/TP4/Hogar_t104.dta"
url_2024 = "https://github.com/JorgeJimenezT/Big-Data-UBA---Jimenez-Menendez-Nunez/raw/main/TP4/usu_hogar_T124.xlsx"

# Descargar y guardar los archivos localmente
file_2004 = "Hogar_t104.dta"
file_2024 = "usu_hogar_T124.xlsx"

# Descargar archivo 2004
response_2004 = requests.get(url_2004)
with open(file_2004, "wb") as f:
    f.write(response_2004.content)

# Descargar archivo 2024
response_2024 = requests.get(url_2024)
with open(file_2024, "wb") as f:
    f.write(response_2024.content)

# Cargar las bases de hogares para 2004 y 2024 usando pyreadstat y pandas
hogar_2004, meta_2004 = pyreadstat.read_dta(file_2004)
hogar_2024 = pd.read_excel(file_2024)

# Estandarizar los nombres de las columnas
hogar_2004.columns = [col.upper() for col in hogar_2004.columns]
hogar_2024.columns = [col.upper() for col in hogar_2024.columns]

# URLs directas de los archivos en GitHub
url_i_2004 = "https://github.com/JorgeJimenezT/Big-Data-UBA---Jimenez-Menendez-Nunez/raw/refs/heads/main/TP3/Individual_t104.dta"
url_i_2024 = "https://github.com/JorgeJimenezT/Big-Data-UBA---Jimenez-Menendez-Nunez/raw/refs/heads/main/TP3/usu_individual_T124.xlsx"

# Leer los archivos directamente desde GitHub
individual_2004 = pd.read_stata(url_i_2004)
individual_2024 = pd.read_excel(url_i_2024)

# Estandarizar los nombres de las columnas en los DataFrames individuales
individual_2004.columns = [col.upper() for col in individual_2004.columns]
individual_2024.columns = [col.upper() for col in individual_2024.columns]

individual_2004['YEAR'] = 2004
individual_2024['YEAR'] = 2024

hogar_2004['YEAR'] = 2004
hogar_2024['YEAR'] = 2024



In [113]:
import pyreadstat
import pandas as pd
import requests



# Diccionario de mapeo: nombres a códigos
aglomerado_mapping = {
    "Gran La Plata": 2,
    "Bahía Blanca - Cerri": 3,
    "Gran Rosario": 4,
    "Gran Santa Fe": 5,
    "Gran Paraná": 6,
    "Posadas": 7,
    "Gran Resistencia": 8,
    "Cdro. Rivadavia – Rada Tilly": 9,
    "Gran Mendoza": 10,
    "Corrientes": 12,
    "Gran Córdoba": 13,
    "Concordia": 14,
    "Formosa": 15,
    "Neuquén – Plottier": 17,
    "S.del Estero - La Banda": 18,
    "Jujuy - Palpalá": 19,
    "Río Gallegos": 20,
    "Gran Catamarca": 22,
    "Salta": 23,
    "La Rioja": 25,
    "San Luis - El Chorrillo": 26,
    "Gran San Juan": 27,
    "Gran Tucumán - T. Viejo": 29,
    "Santa Rosa - Toay": 30,
    "Ushuaia - Río Grande": 31,
    "Ciudad de Buenos Aires": 32,
    "Partidos del GBA": 33,
    "Mar del Plata - Batán": 34,
    "Río Cuarto": 36
}

# Reemplazar nombres por códigos en la columna 'AGLOMERADO'
hogar_2004['AGLOMERADO'] = hogar_2004['AGLOMERADO'].replace(aglomerado_mapping)
hogar_2024['AGLOMERADO'] = hogar_2024['AGLOMERADO'].replace(aglomerado_mapping)


# Filtrar aglomerados correspondientes a Ciudad Autónoma de Buenos Aires o Gran Buenos Aires
ciudad_aglomerados = [32, 33]

hogar_2004_filtered = hogar_2004[hogar_2004["AGLOMERADO"].isin(ciudad_aglomerados)]
hogar_2024_filtered = hogar_2024[hogar_2024["AGLOMERADO"].isin(ciudad_aglomerados)]

individual_2004_filtered = individual_2004[individual_2004["AGLOMERADO"].isin(ciudad_aglomerados)]
individual_2024_filtered = individual_2024[individual_2024["AGLOMERADO"].isin(ciudad_aglomerados)]


# Diagnóstico de las claves antes del merge
print("Diagnóstico antes del merge:")
print("Valores únicos en 'CODUSU' y 'NRO_HOGAR' en hogar_2004:", hogar_2004_filtered[["CODUSU", "NRO_HOGAR"]].nunique())
print("Valores únicos en 'CODUSU' y 'NRO_HOGAR' en individual_2004:", individual_2004[["CODUSU", "NRO_HOGAR"]].nunique())

print("Valores únicos en 'CODUSU' y 'NRO_HOGAR' en hogar_2024:", hogar_2024_filtered[["CODUSU", "NRO_HOGAR"]].nunique())
print("Valores únicos en 'CODUSU' y 'NRO_HOGAR' en individual_2024:", individual_2024[["CODUSU", "NRO_HOGAR"]].nunique())



Diagnóstico antes del merge:
Valores únicos en 'CODUSU' y 'NRO_HOGAR' en hogar_2004: CODUSU       2329
NRO_HOGAR       3
dtype: int64
Valores únicos en 'CODUSU' y 'NRO_HOGAR' en individual_2004: CODUSU       12597
NRO_HOGAR        6
dtype: int64
Valores únicos en 'CODUSU' y 'NRO_HOGAR' en hogar_2024: CODUSU       2540
NRO_HOGAR       4
dtype: int64
Valores únicos en 'CODUSU' y 'NRO_HOGAR' en individual_2024: CODUSU       16005
NRO_HOGAR        5
dtype: int64


In [29]:
print(individual_2004.columns)

Index(['CODUSU', 'NRO_HOGAR', 'COMPONENTE', 'H15', 'ANO4', 'TRIMESTRE',
       'REGION', 'MAS_500', 'AGLOMERADO', 'PONDERA',
       ...
       'IDECCFR', 'RDECCFR', 'GDECCFR', 'PDECCFR', 'ADECCFR', 'PJ1_1', 'PJ2_1',
       'PJ3_1', 'IDIMPP', 'YEAR'],
      dtype='object', length=177)


In [126]:
import pandas as pd

# Definir las columnas relevantes para hogares e individuales
columnas_hogar = ["CODUSU", "NRO_HOGAR", "REGION", "II7", "IV12_3", "IV6", "IV11", "II8", "ITF", "IPCF","II1","IX_TOT"]
columnas_individual = ["CODUSU", "NRO_HOGAR", "CH03", "CH12", "CH06","CH04","NIVEL_ED"]

# Asegurarse de crear copias explícitas
hogar_2004_filtered = hogar_2004[columnas_hogar].copy()
individual_2004_filtered = individual_2004[columnas_individual].copy()

# Convertir NRO_HOGAR a tipo entero antes de transformarlo a cadena
hogar_2004_filtered["NRO_HOGAR"] = hogar_2004_filtered["NRO_HOGAR"].fillna(0).astype(int).astype(str).str.strip()
individual_2004_filtered["NRO_HOGAR"] = individual_2004_filtered["NRO_HOGAR"].fillna(0).astype(int).astype(str).str.strip()

# Eliminar espacios adicionales en CODUSU
hogar_2004_filtered["CODUSU"] = hogar_2004_filtered["CODUSU"].str.strip()
individual_2004_filtered["CODUSU"] = individual_2004_filtered["CODUSU"].str.strip()


# Filtrar las columnas en las bases de hogares
hogar_2024_filtered = hogar_2024[columnas_hogar]

# Filtrar las columnas en las bases individuales
individual_2024_filtered = individual_2024[columnas_individual]

# Realizar el merge entre las bases individuales y de hogares
base_2004_i = pd.merge(individual_2004_filtered, hogar_2004_filtered, on=["CODUSU", "NRO_HOGAR"], how="inner")
base_2024_i = pd.merge(individual_2024_filtered, hogar_2024_filtered, on=["CODUSU", "NRO_HOGAR"], how="inner")

# Unir ambas bases en una sola y agregar el año
base_2004_i["YEAR"] = 2004
base_2024_i["YEAR"] = 2024



In [54]:
print("hogar_2004",hogar_2004_filtered.shape)
print("hogar_2024",hogar_2024_filtered.shape)
print("individual_2004",individual_2004_filtered.shape)
print("individual_2024",individual_2024_filtered.shape)
print("base_2004",base_2004.shape)
print("base_2024",base_2024.shape)

hogar_2004 (12816, 10)
hogar_2024 (16104, 10)
individual_2004 (45289, 4)
individual_2024 (46050, 4)
base_2004 (45289, 13)
base_2024 (46050, 13)



##3.Limpien la base de datos tomando criterios que hagan sentido. Explicar cualquier decisión como el tratamiento de valores faltantes (missing values), extremos (outliers), o variables categóricas. Justifique sus decisiones.  




Valores faltantes: Las filas con valores nulos en las variables clave (ITF, IPCF, etc.) se eliminan porque son esenciales para el análisis.
Outliers: Los valores extremos de ingresos (ITF e IPCF) por encima del percentil 95 se eliminan para evitar sesgos.
Variables categóricas: Las variables categóricas relevantes se convierten en numéricas usando pd.get_dummies para garantizar compatibilidad con modelos estadísticos.
Duplicados: Se eliminan registros duplicados para mantener consistencia en los datos.
División: Finalmente, las bases limpias se separan nuevamente en base_limpia_2004 y base_limpia_2024 para conservar la estructura de los datos originales.

In [127]:
import pandas as pd

# Mostrar información inicial
print(f"Registros iniciales en base_2004: {base_2004_i.shape[0]}")
print(f"Registros iniciales en base_2024: {base_2024_i.shape[0]}")

# Paso 1: Eliminar filas con valores faltantes en variables clave
variables_clave = ["REGION", "II7", "IV12_3", "IV6", "IV11", "II8", "ITF", "IPCF","II1","CH03", "CH12", "CH06","CH04","NIVEL_ED","IX_TOT"]
base_2004 = base_2004_i.dropna(subset=variables_clave)
base_2024 = base_2024_i.dropna(subset=variables_clave)
print(f"Después de eliminar filas con valores faltantes:")
print(f"- base_2004: {base_2004.shape[0]} registros")
print(f"- base_2024: {base_2024.shape[0]} registros")

# Paso 2: Eliminar valores extremos en ITF e IPCF (por encima del percentil 95)
for base, year in zip([base_2004, base_2024], [2004, 2024]):
    itf_percentil_95 = base['ITF'].quantile(0.95)
    ipcf_percentil_95 = base['IPCF'].quantile(0.95)
    base = base[(base['ITF'] <= itf_percentil_95) & (base['IPCF'] <= ipcf_percentil_95)]
    if year == 2004:
        base_2004 = base
    else:
        base_2024 = base
    print(f"Después de eliminar valores extremos en ITF e IPCF (año {year}): {base.shape[0]} registros")

# Paso 3: Convertir variables categóricas relevantes a numéricas (encoding)
variables_categoricas = ['CH04', 'CH12', 'II7', 'IV12_3', 'II8', 'REGION']
base_2004 = pd.get_dummies(base_2004, columns=variables_categoricas, drop_first=True)
base_2024 = pd.get_dummies(base_2024, columns=variables_categoricas, drop_first=True)
print(f"Después de convertir variables categóricas a numéricas:")
print(f"- base_2004: {base_2004.shape[0]} registros, {base_2004.shape[1]} columnas")
print(f"- base_2024: {base_2024.shape[0]} registros, {base_2024.shape[1]} columnas")

# Paso 4: Eliminar duplicados
base_2004 = base_2004.drop_duplicates()
base_2024 = base_2024.drop_duplicates()
print(f"Después de eliminar duplicados:")
print(f"- base_2004: {base_2004.shape[0]} registros")
print(f"- base_2024: {base_2024.shape[0]} registros")

# Guardar las bases limpias
base_2004.to_csv("base_2004_limpia.csv", index=False)
base_2024.to_csv("base_2024_limpia.csv", index=False)
print("Las bases limpias han sido guardadas como 'base_2004_limpia.csv' y 'base_2024_limpia.csv'.")


Registros iniciales en base_2004: 45289
Registros iniciales en base_2024: 46050
Después de eliminar filas con valores faltantes:
- base_2004: 45289 registros
- base_2024: 46050 registros
Después de eliminar valores extremos en ITF e IPCF (año 2004): 42080 registros
Después de eliminar valores extremos en ITF e IPCF (año 2024): 42783 registros
Después de convertir variables categóricas a numéricas:
- base_2004: 42080 registros, 44 columnas
- base_2024: 42783 registros, 42 columnas
Después de eliminar duplicados:
- base_2004: 41967 registros
- base_2024: 42632 registros
Las bases limpias han sido guardadas como 'base_2004_limpia.csv' y 'base_2024_limpia.csv'.


##4.	Construya variables (mínimo 3) que no estén en la base pero que sean relevantes para predecir individuos desocupados (por ejemplo, la proporción de personas que trabajan en el hogar).

Nombre corto: PORC_DEP
Descripción: Calcula el porcentaje de personas menores de 18 años o mayores de 65 años en relación con el total de integrantes del hogar.
Requiere variable adicional: CH04 (edad del individuo) ya debería estar incluida en la base original.

In [131]:
# Crear una copia auxiliar de las bases para trabajar
base_2004_aux = base_2004.copy()
base_2024_aux = base_2024.copy()

# Calcular PORC_DEP: porcentaje de dependientes en el hogar usando CH06 (edad)
# Dependientes: personas con edad < 18 o > 65 años
base_2004_aux["PORC_DEP"] = base_2004_aux.groupby("CODUSU")["CH06"].transform(
    lambda x: ((x < 18) | (x > 65)).sum() / len(x) * 100
)
base_2024_aux["PORC_DEP"] = base_2024_aux.groupby("CODUSU")["CH06"].transform(
    lambda x: ((x < 18) | (x > 65)).sum() / len(x) * 100
)

# Guardar la variable en las bases originales
base_2004["PORC_DEP"] = base_2004_aux["PORC_DEP"]
base_2024["PORC_DEP"] = base_2024_aux["PORC_DEP"]

# Verificar el resultado
print("Base 2004 con PORC_DEP:")
print(base_2004[["CODUSU", "NRO_HOGAR", "CH06", "PORC_DEP"]].head())

print("\nBase 2024 con PORC_DEP:")
print(base_2024[["CODUSU", "NRO_HOGAR", "CH06", "PORC_DEP"]].head())

# Si deseas guardar las bases actualizadas
base_2004.to_csv("base_2004_con_porc_dep.csv", index=False)
base_2024.to_csv("base_2024_con_porc_dep.csv", index=False)

print("Bases actualizadas con PORC_DEP guardadas como 'base_2004_con_porc_dep.csv' y 'base_2024_con_porc_dep.csv'.")



Base 2004 con PORC_DEP:
   CODUSU NRO_HOGAR  CH06  PORC_DEP
0  125098         1  79.0     100.0
1  125397         1  67.0     100.0
2  125515         1  74.0     100.0
3  125558         1  54.0       0.0
4  125558         1  51.0       0.0

Base 2024 con PORC_DEP:
                          CODUSU  NRO_HOGAR  CH06  PORC_DEP
0  TQRMNOPUTHLMKQCDEGGFB00852588          1    44      40.0
1  TQRMNOPUTHLMKQCDEGGFB00852588          1    22      40.0
2  TQRMNOPUTHLMKQCDEGGFB00852588          1     3      40.0
3  TQRMNOPPWHLLKRCDEGGFB00852574          1    54       0.0
4  TQRMNOPPWHLLKRCDEGGFB00852574          1    53       0.0
Bases actualizadas con PORC_DEP guardadas como 'base_2004_con_porc_dep.csv' y 'base_2024_con_porc_dep.csv'.


2. Educación promedio del hogar
Nombre corto: EDU_PROM
Descripción: Se asigna un puntaje a cada nivel educativo (CH12), y se calcula el promedio para cada hogar.
Requiere variable adicional: CH12 ya debería estar incluida en la base original.
Asignar puntajes a los niveles educativos:

1 = Jardín/preescolar
2 = Primario
3 = EGB
4 = Secundario
5 = Polimodal
6 = Terciario
7 = Universitario
8 = Posgrado universitario
9 = Educación especial (0 puntos, ya que no aporta al promedio educativo).

In [134]:
# Asignar puntajes a los niveles educativos en NIVEL_ED
puntajes_educativos = {
    1: 0,  # Primario incompleto
    2: 1,  # Primario completo
    3: 2,  # Secundario incompleto
    4: 3,  # Secundario completo
    5: 4,  # Superior universitario incompleto
    6: 5,  # Superior universitario completo
    7: 0,  # Sin instrucción
    9: 0   # Ns/Nr
}

# Crear una copia auxiliar para trabajar
base_2004_aux = base_2004.copy()
base_2024_aux = base_2024.copy()

# Mapear los puntajes educativos en NIVEL_ED
base_2004_aux["NIVEL_ED_SCORE"] = base_2004_aux["NIVEL_ED"].map(puntajes_educativos)
base_2024_aux["NIVEL_ED_SCORE"] = base_2024_aux["NIVEL_ED"].map(puntajes_educativos)

# Calcular EDU_PROM (promedio educativo por hogar, agrupado por CODUSU)
base_2004_aux["EDU_PROM"] = base_2004_aux.groupby("CODUSU")["NIVEL_ED_SCORE"].transform("mean")
base_2024_aux["EDU_PROM"] = base_2024_aux.groupby("CODUSU")["NIVEL_ED_SCORE"].transform("mean")

# Incorporar la variable EDU_PROM en las bases originales
base_2004["EDU_PROM"] = base_2004_aux["EDU_PROM"]
base_2024["EDU_PROM"] = base_2024_aux["EDU_PROM"]

# Verificar el resultado
print("Base 2004 con EDU_PROM:")
print(base_2004[["CODUSU", "NRO_HOGAR", "NIVEL_ED", "EDU_PROM"]].head())

print("\nBase 2024 con EDU_PROM:")
print(base_2024[["CODUSU", "NRO_HOGAR", "NIVEL_ED", "EDU_PROM"]].head())

# Guardar las bases actualizadas
base_2004.to_csv("base_2004_con_edu_prom.csv", index=False)
base_2024.to_csv("base_2024_con_edu_prom.csv", index=False)

print("Bases actualizadas con EDU_PROM guardadas como 'base_2004_con_edu_prom.csv' y 'base_2024_con_edu_prom.csv'.")


Base 2004 con EDU_PROM:
   CODUSU NRO_HOGAR                         NIVEL_ED  EDU_PROM
0  125098         1            Secundaria Incompleta       NaN
1  125397         1                Primaria Completa       NaN
2  125515         1                Primaria Completa       NaN
3  125558         1                Primaria Completa       NaN
4  125558         1  Superior Universitaria Completa       NaN

Base 2024 con EDU_PROM:
                          CODUSU  NRO_HOGAR  NIVEL_ED  EDU_PROM
0  TQRMNOPUTHLMKQCDEGGFB00852588          1         3      2.40
1  TQRMNOPUTHLMKQCDEGGFB00852588          1         5      2.40
2  TQRMNOPUTHLMKQCDEGGFB00852588          1         7      2.40
3  TQRMNOPPWHLLKRCDEGGFB00852574          1         4      3.75
4  TQRMNOPPWHLLKRCDEGGFB00852574          1         4      3.75
Bases actualizadas con EDU_PROM guardadas como 'base_2004_con_edu_prom.csv' y 'base_2024_con_edu_prom.csv'.


Nombre corto: PERS_X_AMB
Descripción:
Esta variable mide el promedio de personas que habitan por cada ambiente o habitación disponible en el hogar. Se calcula dividiendo el número total de personas en el hogar (IX_TOT) entre el número de ambientes o habitaciones disponibles (II1). El valor resultante refleja la densidad de ocupación de los ambientes en cada hogar.

In [129]:
# Crear una copia auxiliar para trabajar
base_2004_aux = base_2004.copy()
base_2024_aux = base_2024.copy()

# Calcular la variable PERS_X_AMB
base_2004_aux["PERS_X_AMB"] = base_2004_aux["IX_TOT"] / base_2004_aux["II1"]
base_2024_aux["PERS_X_AMB"] = base_2024_aux["IX_TOT"] / base_2024_aux["II1"]

# Incorporar la variable PERS_X_AMB en las bases originales
base_2004["PERS_X_AMB"] = base_2004_aux["PERS_X_AMB"]
base_2024["PERS_X_AMB"] = base_2024_aux["PERS_X_AMB"]

# Verificar el resultado
print("Distribución de la variable PERS_X_AMB en base_2004:")
print(base_2004["PERS_X_AMB"].describe())

print("\nDistribución de la variable PERS_X_AMB en base_2024:")
print(base_2024["PERS_X_AMB"].describe())

# Guardar las bases actualizadas
base_2004.to_csv("base_2004_con_pers_x_amb.csv", index=False)
base_2024.to_csv("base_2024_con_pers_x_amb.csv", index=False)

print("Bases actualizadas con PERS_X_AMB guardadas como 'base_2004_con_pers_x_amb.csv' y 'base_2024_con_pers_x_amb.csv'.")



Distribución de la variable PERS_X_AMB en base_2004:
count    4.196700e+04
mean              inf
std               NaN
min      1.010101e-02
25%      1.000000e+00
50%      1.500000e+00
75%      2.000000e+00
max               inf
Name: PERS_X_AMB, dtype: float64

Distribución de la variable PERS_X_AMB en base_2024:
count    4.263200e+04
mean              inf
std               NaN
min      1.250000e-01
25%      8.333333e-01
50%      1.250000e+00
75%      1.666667e+00
max               inf
Name: PERS_X_AMB, dtype: float64
Bases actualizadas con PERS_X_AMB guardadas como 'base_2004_con_pers_x_amb.csv' y 'base_2024_con_pers_x_amb.csv'.


##5.	Presenten estadísticas descriptivas de tres variables de la encuesta de hogar que ustedes creen que pueden ser relevantes para predecir la desocupación. Comenten las estadísticas obtenidas.

In [136]:
# Seleccionar las variables relevantes
variables_hogar = ["CH06", "PORC_DEP", "ITF"]

# Generar estadísticas descriptivas para cada variable
def describir_variable(base, variable, nombre_variable):
    print(f"\n--- Estadísticas descriptivas para {nombre_variable} ({variable}) ---")
    print(base[variable].describe(percentiles=[0.25, 0.5, 0.75]))
    if base[variable].dtype == "object" or base[variable].nunique() <= 10:
        print("\nFrecuencias:")
        print(base[variable].value_counts(normalize=True) * 100)

# Estadísticas para base_2004
print("=== Estadísticas descriptivas: Base 2004 ===")
for var in variables_hogar:
    describir_variable(base_2004, var, var)

# Estadísticas para base_2024
print("\n=== Estadísticas descriptivas: Base 2024 ===")
for var in variables_hogar:
    describir_variable(base_2024, var, var)


=== Estadísticas descriptivas: Base 2004 ===

--- Estadísticas descriptivas para CH06 (CH06) ---
count     41967.0
unique       99.0
top          12.0
freq        892.0
Name: CH06, dtype: float64

--- Estadísticas descriptivas para PORC_DEP (PORC_DEP) ---
count    41967.000000
mean        42.180761
std         25.908060
min          0.000000
25%         25.000000
50%         50.000000
75%         60.000000
max        100.000000
Name: PORC_DEP, dtype: float64

--- Estadísticas descriptivas para ITF (ITF) ---
count    41967.000000
mean       871.351443
std        614.841803
min          0.000000
25%        400.000000
50%        700.000000
75%       1200.000000
max       3000.000000
Name: ITF, dtype: float64

=== Estadísticas descriptivas: Base 2024 ===

--- Estadísticas descriptivas para CH06 (CH06) ---
count    42632.000000
mean        36.083482
std         22.308901
min         -1.000000
25%         17.000000
50%         34.000000
75%         53.000000
max        101.000000
Name: CH06,